# Gaussian Process and Neural Network Gaussian Processes

This notebook is for looking at traditional gaussian processes and their neural network implementation for use in optimal control and state prediction. Various parameters are test on a real dataset

## Import libraries and Dataset

In [1]:
import matplotlib.pyplot as plt
from nngp_tf2 import nngp
import numpy as np
from sklearn import gaussian_process
import tensorflow as tf
from thesis.data import blackbird_dataset as rbd
import time

In [2]:
# Generate Test Dataset
test_df = rbd.cleaned_blackbird_test(
    'figure8', 'Constant', 0.5
)

In [3]:
# Set up downsampling
downsample_dict={
        'stride_pos': 36,          # 10 Hz
        'stride_att': 36,          # 10 Hz
        'stride_pos_ref': 19,      # 10 Hz
        'stride_att_ref': 19,      # 10 Hz
        'stride_motor_speeds': 19, # 10 Hz
        'stride_accel': 10,        # 10 Hz
        'stride_gyro': 10,         # 10 Hz
        'stride_pwms': 9,           # 20 Hz
    }

# Get optimal control input/output
X, Y, tvec_y, info = rbd.generate_opt_control_test_data(
    test_df,
    past_delta_t=2,
    future_delta_t=1,
    downsample_dict=downsample_dict
)

for ko, kd in zip(info.keys(), downsample_dict.keys()):
    n, dt = info[ko]
    d = downsample_dict[kd]
    print("\t%s: %i [samples]\t%f [Hz]" %(ko, int(np.ceil(n/d)), 1./(dt*d)))

	vicon position: 20 [samples]	9.999200 [Hz]
	vicon attitude: 20 [samples]	9.999200 [Hz]
	reference position: 10 [samples]	9.919257 [Hz]
	reference attitude: 10 [samples]	9.919257 [Hz]
	motor speeds: 20 [samples]	9.889436 [Hz]
	accelerometer: 20 [samples]	10.000000 [Hz]
	gyroscope: 20 [samples]	10.000000 [Hz]
	PWM: 21 [samples]	20.881622 [Hz]


In [4]:
# Describing the data
print("Number of test points: %i" % X.shape[0])
print("Input dimension: %i" % X.shape[1])
print("Output dimension: %s" % Y.shape[1:])

Number of test points: 38089
Input dimension: 410
Output dimension: 84


## Gaussian Processes

Gaussian processes are fairly well known and established and starts with the standard multivariate gaussian distribution with the probability denisty function of the $k$-state vector $\vec{x}$ with a mean vector $\mu_x$ and a covariance matrix $\Sigma$:

$$p(\vec{x}) \sim \mathcal{N}\left(\vec{\mu}, \Sigma \right) = \frac{1}{\sqrt{\left(2\pi\right)^k \left|\Sigma\right|}}exp\left(-\frac{1}{2}(\vec{x} - \mu_x)^T \Sigma^{-1} (\vec{x} - \mu)\right)$$

However, it is possible to split $k$ states into two portions still based on the multivariate gaussian distribution.

$$p(\vec{x}, \vec{y}) \sim \mathcal{N}\left(\begin{bmatrix}\mu_x\\ \mu_y\end{bmatrix}, \begin{bmatrix}\Sigma_{xx}&\Sigma_{xy}\\\Sigma_{yx}&\Sigma_{yy}\end{bmatrix}\right)$$

If the portion of $y$ of the original $k$ states is known, then it is possible to form a new probability distribution of the unknown portion $x$:

$$p(\vec{x}|\vec{y}) \sim \mathcal{N} \left(\mu_x + \Sigma_{xy}\Sigma_{yy}^{-1}(\vec{y} - \mu_y), \Sigma_{xx} - \Sigma_{xy}\Sigma_{yy}^{-1}\Sigma_{xy}^T\right)$$

Gaussian processes extends this notion by allowing for the mean to be a function and the covariance matrix becomes a kernel function.

$$p(\vec{x}) \sim \mathcal{GP}\left(\mu(\vec{x}), k(\vec{x},\vec{x}')\right)$$

In [5]:
# Generate some test and train points
n_stride = 1000
Xtrain, Ytrain = X[::n_stride], Y[::n_stride]
Xtest, Ytest = X[(n_stride//2)::n_stride], Y[(n_stride//2)::n_stride]

print("N_train: %i" % Xtrain.shape[0])
print("N_test: %i" % Xtest.shape[0])

N_train: 39
N_test: 38


In [6]:
kernel_list = [
    gaussian_process.kernels.DotProduct,
    gaussian_process.kernels.ExpSineSquared,
    gaussian_process.kernels.Matern,
    gaussian_process.kernels.RBF,
    gaussian_process.kernels.RationalQuadratic,
]

kernel_mse = [0.]*len(kernel_list)

for i, kernel in enumerate(kernel_list):
    # Have a kernel with a little bit of white noise
    adj_kernel = (
        kernel() +
        gaussian_process.kernels.WhiteKernel(0.001)
    )

    gpr = gaussian_process.GaussianProcessRegressor(
        kernel=adj_kernel,
        random_state=42,  # As it should be
        n_restarts_optimizer=10
    )# .fit(Xtrain, Ytrain)
    
    t1 = time.time()
    
    gpr.fit(Xtrain, Ytrain)  # Optimized parameters in kernel_
    
    t2 = time.time()
    
    # Store MSE in history vector
    kernel_mse[i] = np.sum((Ytest - gpr.predict(Xtest)) ** 2)/Xtest.shape[0]
    
    t3 = time.time()
    
    # Print out results
    kparams = gpr.kernel_.get_params()
    print("Kernel %i" % i,
          "\n\tTime Training: %f [s]" % (t2 - t1),
          "\n\tTime Predicting: %f" % (t3 - t2),
          "\n\tKernel: %s + %s" % (kparams['k1'], kparams['k2']),
          "\n\tMSE: %f" % kernel_mse[i]
         )

Kernel 0 
	Time Training: 0.207093 [s] 
	Time Predicting: 0.000000 
	Kernel: DotProduct(sigma_0=1.61e-05) + WhiteKernel(noise_level=1e-05) 
	MSE: 0.012945
Kernel 1 
	Time Training: 0.464254 [s] 
	Time Predicting: 0.000995 
	Kernel: ExpSineSquared(length_scale=0.00287, periodicity=1e+05) + WhiteKernel(noise_level=1.86e-05) 
	MSE: 0.012931
Kernel 2 
	Time Training: 0.291136 [s] 
	Time Predicting: 0.000999 
	Kernel: Matern(length_scale=118, nu=1.5) + WhiteKernel(noise_level=2.4e-05) 
	MSE: 0.013219
Kernel 3 
	Time Training: 0.249315 [s] 
	Time Predicting: 0.001000 
	Kernel: RBF(length_scale=45.7) + WhiteKernel(noise_level=1.86e-05) 
	MSE: 0.012931
Kernel 4 
	Time Training: 1.596073 [s] 
	Time Predicting: 0.001001 
	Kernel: RationalQuadratic(alpha=1e+05, length_scale=45.7) + WhiteKernel(noise_level=1.86e-05) 
	MSE: 0.012931


C:\Users\Patrick\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00036709,  0.10830282, -0.0036593 ]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 104, 'nit': 29, 'warnflag': 2}
  ConvergenceWarning)


## Neural Network Gaussian Processes

Neural Network Gaussian Process comes from the paper [Deep Neural Networks as Gaussian Processes](https://arxiv.org/pdf/1711.00165.pdf). Originally the authors has [a GitHub repo](https://github.com/brain-research/nngp) for Tensorflow 1.x but was later implemented in [this GitHub repo](https://github.com/erees1/NNGP) for Tensorflow 2.x. The network is implemented as the following at the $l$th layer with a layer width $N_l$ using the nonlinear function $\phi$:

$$z_i^l(x) = b_i^l + \sum_{j=1}^{N_l} W_{ij}^l x_j^l (x),\quad x_j^l (x) = \phi(z_j^{l-1}(x))$$

Both the bias vector $b_i^l$ and the weight matrix $W_{ij}^l$ are condisered independent and randomly drawn from zero mean distributions with variances of $\sigma_b^2$ and $\sigma_w^2 / N_l$ respectively. The paper asymptotic runtime is  $\mathcal{O}\left(n_g^2 n_v n_c + L\left(n_\text{train}^2 + n_\text{train} n_\text{test}\right)\right)$ where:

* $n_g^2$ is sampling density for pair of Gaussian random variables.
* $n_v$ is the sampling density for variance in variance and correlation grid.
* $n_c$ is the sampling density for correlation in variance and correlation grid.
* $L$ is the number of layers.
* $n_\text{train}$ and $n_\text{test}$ are the train and test set sizes.

The hyperparameters needed to be optimized are the kernel parameters $\sigma_b$ and $\sigma_w$ which are the standard deviations of the bias vector and the weight matrix respectively.


In [7]:
import time

# Set up kernel parameters
act = tf.nn.relu
n_layers = 5

sigma_b_search = 10. ** np.linspace(-2, -1, 11)
sigma_w_search = 10. ** np.linspace(-2, 1, 11)
mse_nngp = np.zeros(sigma_b_search.shape + sigma_w_search.shape)

tstart = time.time()

# Create the kernel
for i, sb in enumerate(sigma_b_search):
    for j, sw in enumerate(sigma_w_search):
        general_kernel = nngp.GeneralKernel(act,
                                            L=n_layers,
                                            n_g=101,  # 401,
                                            n_v=100,  # 400,
                                            n_c=100,  # 400,
                                            u_max=10,
                                            s_max=100,
                                            sigma_b=sb,
                                            sigma_w=sw,
                                            save_loc='../nngp_tf2/kernel_grids')
        try:
            # Train and get prediction
            mu_bar, K_bar = nngp.GP_cholesky(
                Xtrain,
                Ytrain,
                Xtest,
                general_kernel.K
            )

            # Determin NNGP MSE loss
            nngp_loss = np.sum((Ytest - mu_bar) ** 2)/Xtrain.shape[0]
            print(nngp_loss)
            mse_nngp[i,j] = nngp_loss
        except:
            mse_nngp[i,j] = np.nan

        del general_kernel
            
tstop = time.time()

# Print run time
print("Time to train: %f [s]" % (tstop - tstart))

2021-02-16 13:00:40,709 - Loaded grid from file


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'


2021-02-16 13:00:40,779 - AutoGraph could not transform <function _call_general_K at 0x000001C5A84422F0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'


2021-02-16 13:00:40,790 - AutoGraph could not transform <function _normalize at 0x000001C5A8443598> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'


...

0.029034618232652513


2021-02-16 13:04:11,160 - 11 out of the last 11 calls to <function _call_general_K at 0x000001C5A84422F0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.


2021-02-16 13:04:11,760 - 11 out of the last 11 calls to <function _call_general_K at 0x000001C5A84422F0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.


2021-02-16 13:04:12,361 - 11 out of the last 11 calls to <function _call_general_K at 0x000001C5A84422F0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.


0.030236759303355936
Time to train: 211.665529 [s]


In [8]:
print("Minimum MSE: %f" % np.nanmin(mse_nngp))
ind = np.unravel_index(np.nanargmin(mse_nngp, axis=None), mse_nngp.shape)
print("Min args:\n\tsimga_b = %f\n\tsigma_w = %f" % (
    sigma_b_search[ind[0]],
    sigma_w_search[ind[1]]
))

Minimum MSE: 0.014489
Min args:
	simga_b = 0.063096
	sigma_w = 0.630957
